# Graph Visualization

<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/notebooks/networkViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>

## requirements.txt


In [ ]:
!pip install -q minio
!pip install -q kglab
!pip install -q sparqlwrapper
!pip install -q pygraphml
!pip install -q ipysigma

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import kglab
from minio import Minio
from rdflib import Graph, plugin
import plotly.express as px
import pandas as pd
from urllib.request import urlopen
import os,json

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma
from SPARQLWrapper import SPARQLWrapper, JSON
from hashlib import sha256

In [2]:
def hash_text(text):
    return sha256(text.encode()).hexdigest()[:8]

In [3]:
## Remote SPARQL server
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



In [28]:
rp1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType ?sName ?tName
WHERE {
  ?source a ?sType .
  OPTIONAL { ?source schema:name ?sName . }
  
  ?target a ?tType .
  OPTIONAL { ?target schema:name ?tName .}
  
   ?source ?type ?target .

}
"""

ep = "http://0.0.0.0:7878/query"

df = get_sparql_dataframe(ep, rp1)


In [29]:
df.to_parquet('inspiretypes.parquet')
# df = pd.read_parquet('path/to/your/file.parquet')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  107 non-null    object
 1   type    107 non-null    object
 2   target  107 non-null    object
 3   sType   107 non-null    object
 4   tType   107 non-null    object
 5   sName   67 non-null     object
 6   tName   50 non-null     object
dtypes: object(7)
memory usage: 6.0+ KB


In [31]:
df.head(10)

,source,type,target,sType,tType,sName,tName
0,865d031934f04810ee2f50aea7dfe223,https://schema.org/softwareHelp,c4d9fac06ed20cdfc6eaf4882d43cccf,https://schema.org/SoftwareApplication,https://schema.org/CreativeWork,HADES (Health Analytics Data-To-Evidence Suite),The Book of OHDSI: Chapter 13
1,865d031934f04810ee2f50aea7dfe223,https://aphrc.org/analysisModelDetails,62149e76f9f653b1171454f47df55073,https://schema.org/SoftwareApplication,https://schema.org/ItemList,HADES (Health Analytics Data-To-Evidence Suite),AnalysisModelDetails
2,https://example.org/id/concept/317009,https://schema.org/increasesRiskOf,4d9a94d2536367cfcffd0309c4656a7,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCondition,Asthma,None
3,https://example.org/id/concept/317009,https://schema.org/code,881cc83b6e6490f6404ce44106eec64d,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCode,Asthma,None
4,https://example.org/id/concept/201820,https://schema.org/increasesRiskOf,337667295d08473627f442bc15c84963,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCondition,Diabetes mellitus,None
5,https://example.org/id/concept/201820,https://schema.org/code,297487b9ecb0acaf24406a2c4b2be0b6,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCode,Diabetes mellitus,None
6,120066d45c236713bb9ac579bab8aa5,https://schema.org/item,b2e5488b06e2e6bfbc8d2cc61374ed4,https://schema.org/ListItem,https://schema.org/SoftwareApplication,HADES,None
7,1175f9ab433a1866bfd4b82cebfd251d,https://schema.org/item,62ae6c73e2fe652e322605c77d15949c,https://schema.org/ListItem,https://schema.org/ItemList,OHDSIStudyPackageAggregateData,IndicatorCollections
8,502b2ccbc4dae9263d175ff5cd2e0754,https://schema.org/item,1d04821ed4c18279bbac6775b5094226,https://schema.org/ListItem,https://schema.org/SoftwareApplication,WebAPI,None
9,51d5b6c5a0f18d6edfa85714f84d0c95,https://schema.org/item,8e5ad480369ac1c9ddace45213f5c895,https://schema.org/ListItem,https://schema.org/ItemList,OHDSIStudyPackageMetadata,MachineActionableSettingByAnalysis


In [32]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!  
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])
g = GraphML()
g.directed = False

for index, row in df.iterrows():
    n1 = g.add_node(row['source'])
    n1['type'] = row['sType']
    n1['name'] = row['sName']
    
    n2 = g.add_node(row['target'])
    n2['type'] = row['tType']
    n2['name'] = row['tName']

    e1 = g.add_edge(n1, n2)
    
fname = "./inspire.graphml"
parser = GraphMLParser()
parser.write(g, fname)

In [33]:
# g = nx.les_miserables_graph()
g = nx.read_graphml("./inspire.graphml")

In [34]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)



Sigma(nx.Graph with 114 nodes and 107 edges)